# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port elizabeth,-33.9180,25.5701,84.09,66,0,11.50,ZA,1706538451
1,1,taoudenni,22.6783,-3.9836,84.18,7,0,9.10,ML,1706538623
2,2,new norfolk,-42.7826,147.0587,51.30,95,15,2.95,AU,1706538623
3,3,arraial do cabo,-22.9661,-42.0278,82.36,74,75,6.91,BR,1706538623
4,4,margaret river,-33.9500,115.0667,58.37,79,0,12.17,AU,1706538624


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    #frame_width = 700,
    #frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    color = "City"
)

# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [6]:
# Narrow down cities that fit criteria and drop any results with null values
temp_criteria = city_data_df['Max Temp'] > 21.00 
cloud_criteria = city_data_df['Cloudiness'] == 0
wind_criteria = city_data_df['Wind Speed'] < 4.50
criteria = temp_criteria & cloud_criteria & wind_criteria & humidity_criteria

# Use .loc to create a copy of the DataFrame
narrowed_city_data_df = city_data_df.loc[criteria].copy()

# Drop any rows with null values
narrowed_city_data_df.dropna(inplace=True)

# Display sample data
narrowed_city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
14,14,tura,25.5198,90.2201,57.49,57,0,3.06,IN,1706538625
26,26,blackmans bay,-43.0167,147.3167,50.13,78,0,1.99,AU,1706538628
169,169,brookings,44.3114,-96.7984,31.12,93,0,0.00,US,1706538356
198,198,la palma,33.8464,-118.0467,68.50,66,0,3.44,US,1706538513
327,327,turkmenabat,39.0733,63.5786,40.98,87,0,2.30,TM,1706538683
347,347,corsicana,32.0954,-96.4689,48.16,81,0,3.44,US,1706538686
351,351,ridgecrest,35.6225,-117.6709,39.06,93,0,0.00,US,1706538686
473,473,boonville,38.9736,-92.7432,36.63,100,0,0.00,US,1706538706
555,555,puerto san jose,13.9256,-90.8244,84.87,52,0,3.18,GT,1706538720


### Step 3: Create a new DataFrame called `hotel_df`.

In [7]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
columns_to_copy = ['City', 'Country', 'Lat', 'Lng', 'Humidity']
hotel_df = narrowed_city_data_df[columns_to_copy].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
14,tura,IN,25.5198,90.2201,57,
26,blackmans bay,AU,-43.0167,147.3167,78,
169,brookings,US,44.3114,-96.7984,93,
198,la palma,US,33.8464,-118.0467,66,
327,turkmenabat,TM,39.0733,63.5786,87,
347,corsicana,US,32.0954,-96.4689,81,
351,ridgecrest,US,35.6225,-117.6709,93,
473,boonville,US,38.9736,-92.7432,100,
555,puerto san jose,GT,13.9256,-90.8244,52,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [8]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "radius": radius,
    "type": "accommodation.hotel",
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    Lat = row['Lat']
    Lng = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{Lng},{Lat},{radius}"
    params["bias"] = f"proximity:{Lng},{Lat}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
tura - nearest hotel: No hotel found
blackmans bay - nearest hotel: Villa Howden
brookings - nearest hotel: Quality Inn
la palma - nearest hotel: La Quinta Inn & Suites by Wyndham Buena Park
turkmenabat - nearest hotel: Yupek Yoly
corsicana - nearest hotel: Comfort Inn
ridgecrest - nearest hotel: Best Western China Lake Inn
boonville - nearest hotel: Hotel Frederick
puerto san jose - nearest hotel: Martita


,City,Country,Lat,Lng,Humidity,Hotel Name
14,tura,IN,25.5198,90.2201,57,No hotel found
26,blackmans bay,AU,-43.0167,147.3167,78,Villa Howden
169,brookings,US,44.3114,-96.7984,93,Quality Inn
198,la palma,US,33.8464,-118.0467,66,La Quinta Inn & Suites by Wyndham Buena Park
327,turkmenabat,TM,39.0733,63.5786,87,Yupek Yoly
347,corsicana,US,32.0954,-96.4689,81,Comfort Inn
351,ridgecrest,US,35.6225,-117.6709,93,Best Western China Lake Inn
473,boonville,US,38.9736,-92.7432,100,Hotel Frederick
555,puerto san jose,GT,13.9256,-90.8244,52,Martita


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
%%capture --no-display

# Configure the map plot
hover_cols = ["City", "Hotel Name", "Country", "Humidity"]

map_plot_2 = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 0.9,
    color = "City",
    hover_cols=hover_cols,
)

# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)